In [1]:
from nltk.tokenize.treebank import TreebankWordDetokenizer as Detok


class Detokenizer(object):
    '''
    'Detokenize' is a stateless class that contains a single static method 'detokenize'
    '''

    @staticmethod
    def __switch(tokens, find, replace):
        new_list = []

        iterator = iter(tokens)
        for token in iterator:
            if token == find:
                token = replace

            new_list.append(token)

        return new_list

    @staticmethod
    def __box_forward(tokens, find, replace):
        new_list = []

        iterator = iter(tokens)
        for token in iterator:
            if token == find:
                token = replace + next(iterator)

            new_list.append(token)

        return new_list

    @staticmethod
    def __box_backwards(tokens, find, replace):

        new_list = []

        for token in tokens:
            if token != find:
                new_list.append(token)

            else:

                if len(new_list) > 0:
                    new_list[-1] = new_list[-1] + replace
                # otherwise just ignore - not much to be done ...

        return new_list

    @staticmethod
    def detokenize(tokens):
        '''
        '''
        new_tokens = tokens

        new_tokens = Detokenizer.__box_forward(new_tokens, '-LRB-', '(')
        new_tokens = Detokenizer.__box_forward(new_tokens, '``', '"')

        new_tokens = Detokenizer.__box_backwards(new_tokens, '-RRB-', ')')
        new_tokens = Detokenizer.__box_backwards(new_tokens, '\'\'', '"')

        new_tokens = Detokenizer.__switch(new_tokens, '--', '-')

        detokenizer = Detok()
        return detokenizer.detokenize(new_tokens)


In [2]:
import ijson
import csv


In [4]:
output = open('out.csv', 'w', encoding='utf-8', newline='')
csv_out = csv.writer(output, lineterminator='\n')
fieldnames = ['tac_doc_id', 
              'tac_sentence_id', 
              'sentence', 
              'indexed_tokens', 
              'subj_start', 
              'subj_end', 
              'obj_start', 
              'obj_end',
              'assessment',
              'alt_subj_start', 
              'alt_subj_end', 
              'alt_obj_start', 
              'alt_obj_end']
csv_out.writerow(fieldnames)


input_stream = open(r'C:\Users\jyellin\re_1\tacred\data\json\test.json', encoding='utf-8')
json_stream = ijson.items(input_stream, 'item')
detokenizer = Detokenizer()
for item in filter(lambda item: item['relation'] == 'per:cities_of_residence', json_stream):
    tokens = item['token']
    sentence = detokenizer.detokenize(tokens)
    indexed_tokens = [(i, token) for i, token in enumerate(tokens) ]
    subj_start = item['subj_start']
    subj_end = item['subj_end']
    obj_start = item['obj_start']
    obj_end = item['obj_end']
    doc_id =  item['docid']
    sentence_id = item['id']
    assesment = None
    alt_subj_start = None
    alt_subj_end = None
    alt_obj_start = None
    alt_obj_end = None
    
    
    
    csv_out.writerow([doc_id, 
                     sentence_id, 
                     sentence, 
                     indexed_tokens, 
                     subj_start,
                     subj_end, 
                     obj_start,
                     obj_end,
                     assesment,
                     alt_subj_start,
                     alt_subj_end,
                     alt_obj_start, 
                     alt_obj_end])
                     
                     
                     
output.close()